In [1]:
import os.path as path


benchmarks_location = 'benchmarks'
file_name_template = 'p0%s_%s.txt'
indices = range(1,8)
categories = ['c', 'w', 'p', 's']

In [2]:
import requests


dataset_location = 'https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/'
for index in indices:
    for category in categories:
        filename = file_name_template % (index, category)
        r = requests.get(dataset_location + filename, allow_redirects=True)
        file = open(path.join(benchmarks_location, filename), 'wb')
        file.write(r.content)
        file.close()

In [2]:
from ptas import WEIGHT, VALUE


def make_item(weight, value):
    item = [-1, -1]
    item[WEIGHT] = weight
    item[VALUE] = value
    return tuple(item)


CAPACITY = 0
ITEMS = 1
OPTIMAL = 2
dataset = []
for index in indices:
    f = open(path.join(benchmarks_location, file_name_template % (index, 'c')))
    capacity = int(f.read())
    f.close()
    f = open(path.join(benchmarks_location, file_name_template % (index, 'w')))
    weights = [int(line) for line in f.readlines()]
    f.close()
    f = open(path.join(benchmarks_location, file_name_template % (index, 'p')))
    profits = [int(line) for line in f.readlines()]
    f.close()
    f = open(path.join(benchmarks_location, file_name_template % (index, 's')))
    binary_optimal = [int(line) for line in f.readlines()]
    optimal = []
    for i in range(len(binary_optimal)):
        if binary_optimal[i] == 1:
            optimal.append(i)
    f.close()
    items = [make_item(weight=weights[i], value=profits[i]) for i in range(len(weights))]
    sample = [-1, -1, -1]
    sample[CAPACITY] = capacity
    sample[ITEMS] = items
    sample[OPTIMAL] = optimal
    dataset.append(tuple(sample))
#dataset

In [6]:
from ptas import sahni_ptas; k=3 # на этом наборе данных лучше результаты получаются с k=3
from dynamic_programming import knapsack_dp

algorithms = [sahni_ptas, knapsack_dp]

In [7]:
import time as timer


def get_output_and_time(iterations, f, *args):
    start = timer.time()
    for i in range(iterations):
        output = f(*args)
    elapsed = timer.time() - start
    return *output, elapsed

In [9]:
for index in range(len(dataset)):
    print('Sample %s' % (index+1), end='\n\n')
    sample = dataset[index]
    for algorithm in algorithms:
        print(algorithm.__name__)
        profit = 0
        if algorithm.__name__ == 'sahni_ptas':
            iters = 100
            profit, items, comparisons, time = get_output_and_time(iters, algorithm, sample[CAPACITY], sample[ITEMS], k)
            binary_items = []
            for item_index in range(len(sample[ITEMS])):
                if item_index in items:
                    binary_items.append(1)
                else:
                    binary_items.append(0)
            items = binary_items
        elif algorithm.__name__ == 'knapsack_dp':
            weights = []
            prices = []
            for item in sample[ITEMS]:
                weights.append(item[WEIGHT])
                prices.append(item[VALUE])
            items, comparisons, time = get_output_and_time(iters, algorithm, sample[CAPACITY], weights, prices)
            for item_index in range(len(items)):
                if items[item_index] == 1:
                    profit += sample[ITEMS][item_index][VALUE]
        weight = 0
        for item_index in range(len(items)):
            if items[item_index] == 1:
                weight += sample[ITEMS][item_index][WEIGHT]
        print('Items: %s\nProfit: %s\nWeight: %s\nComparisons: %s\nTime: %s (%s iterations)'\
              % (items, profit, weight, comparisons, time, iters), end='\n\n')
    print('')

Sample 1

sahni_ptas
Items: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
Profit: 309
Weight: 165
Comparisons: 187
Time: 0.03499937057495117 (100 iterations)

knapsack_dp
Items: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
Profit: 309
Weight: 165
Comparisons: 6446
Time: 0.2720215320587158 (100 iterations)


Sample 2

sahni_ptas
Items: [0, 1, 1, 1, 0]
Profit: 51
Weight: 26
Comparisons: 12
Time: 0.001999378204345703 (100 iterations)

knapsack_dp
Items: [0, 1, 1, 1, 0]
Profit: 51
Weight: 26
Comparisons: 553
Time: 0.022003889083862305 (100 iterations)


Sample 3

sahni_ptas
Items: [1, 1, 0, 0, 1, 0]
Profit: 150
Weight: 190
Comparisons: 32
Time: 0.004996538162231445 (100 iterations)

knapsack_dp
Items: [1, 1, 0, 0, 1, 0]
Profit: 150
Weight: 190
Comparisons: 4622
Time: 0.18700289726257324 (100 iterations)


Sample 4

sahni_ptas
Items: [1, 1, 0, 0, 0, 1, 1]
Profit: 105
Weight: 50
Comparisons: 61
Time: 0.01203775405883789 (100 iterations)

knapsack_dp
Items: [1, 0, 0, 1, 0, 0, 0]
Profit: 107
Weight: 50
Comparisons: 1445
T